# Setting Up

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -q onnx
!pip install -q onnxruntime
!pip install -q onnxruntime-gpu
!pip install -q onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.3 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [5]:
from tqdm.notebook import tqdm
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/Capstone_3

/content/drive/MyDrive/Capstone_3


In [8]:
df = pd.read_csv("/content/drive/MyDrive/Capstone_3/Stack_Overflow_data.csv")
df.head()

,Question,Summary,Tags,Username
0,IntelliJ IDEA and Glassfish for Java web app,I am having trouble getting a simple java web app running. I am going through this tutorial. I am using: IntelliJ IDEA ultimate ed. 2023.3.2 JDK: 19.0.1 Glassfish: 7.0.9 I have gotten to part 5 under ...,"['java', 'intellij-idea', 'permissions', 'glassfish']",Krabs
1,How to style Menu item in Ng Prime Menubar?,"i just started using ng prime, i used the p-menubar <p-menubar [model]=""items""></p-menubar> this is the component file : this.items = [ { label: 'First item', ...","['angular', 'primeng']",yacine ben gharsallah
2,How does this function not return true?,"I am writing a program in Python and so far everthing seems working correctly. I am a bit confused by only one result in a series of mathematics, in the case of x = 169, i = 13 this function seems to ...","['python', 'python-3.x', 'mpmath']",Willtech
3,Error in setting cell values for a TableView column,I'm a newbie to JavaFX (still learning) and I'm currently in the topic of TableViews. I have followed this one particular YouTube video where he made a program about inserting cells to a TableView and ...,"['java', 'javafx']",Doggoder
4,Updating the coordinates of the child element after scaling,"Help me figure out how to update coordinates for child elements in the form of imgElement images in div canvas-cameras. In the center there is a div with a background picture. In the div, I add as ...","['javascript', 'html', 'css']",Snacker991


In [9]:
df = df.dropna().reset_index(drop=True)
df.shape

(29948, 4)

In [10]:
tags_list = df.Tags.to_list()
tags_count = {}
for Tags in tags_list:
  tags_list = eval(Tags)
  for Tags in tags_list:
    if Tags in tags_count.keys():
      tags_count[Tags] += 1
    else:
      tags_count[Tags] = 1
print(f"Number of Tags: {len(tags_count)}")
print(tags_count)

threshold = int(len(df) * 0.01)
rare_tags = [key for key, value in tags_count.items() if value < threshold]
len(rare_tags)

Number of Tags: 11970
{'java': 1485, 'intellij-idea': 69, 'permissions': 23, 'glassfish': 7, 'angular': 539, 'primeng': 13, 'python': 3769, 'python-3.x': 480, 'mpmath': 2, 'javafx': 46, 'javascript': 2614, 'html': 1025, 'css': 746, 'react-native': 439, 'api': 3, 'mobile': 72, 'google-cloud-translate': 1, 'google-sheets': 160, 'xml-import': 1, 'adal': 1, 'msal': 13, 'c#': 1307, 'playwright': 42, 'julia': 15, 'extract': 8, 'data-processing': 1, 'cifar100': 1, 'ios': 478, 'swiftui': 263, 'visibility': 4, 'swiftui-list': 8, 'ansible': 42, 'ansible-automation-platform': 1, 'selenium-webdriver': 215, 'webdriver': 16, 'r': 676, 'shiny': 67, 'reactable': 2, 'android': 1180, 'android-studio': 156, 'gradle': 127, 'android-gradle-plugin': 17, 'build.gradle': 22, 'networking': 68, 'raspberry-pi': 40, 'snmp': 3, 'mib': 1, 'unit-testing': 86, 'go': 163, 'vue.js': 213, 'browser': 37, 'camera': 36, 'object-detection': 18, 'tensorflow.js': 8, 'django': 413, 'django-views': 49, 'mixins': 1, 'azure': 353

11934

In [11]:
tags_list = df.Tags.to_list()
revised_tags_list = []
indices_to_drop = []

for idx, Tags in enumerate(tags_list):
  tags_list = eval(Tags)
  revised_tags = []

  for Tags in tags_list:
    if Tags not in rare_tags:
      revised_tags.append(Tags)

  if len(revised_tags) == 0:
    indices_to_drop.append(idx)
  else:
    revised_tags_list.append(revised_tags)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_tags'] = revised_tags_list
df.shape

(20262, 5)

In [12]:
revised_tags_list = df.revised_tags.to_list()
revised_tags_count = {}
for Tags in revised_tags_list:
  tags_list = Tags
  for Tags in tags_list:
    if Tags in revised_tags_count.keys():
      revised_tags_count[Tags] += 1
    else:
      revised_tags_count[Tags] = 1
print(f"Number of Tags: {len(revised_tags_count)}")

Number of Tags: 36


In [13]:
encode_tags_types = { key: idx for idx, (key, value) in enumerate(revised_tags_count.items())}
with open("tags_types_encoded.json", "w") as fp:
  json.dump(encode_tags_types, fp)

# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_tags_list = []
revised_tags_list = df.revised_tags.to_list()

for revised_tags in revised_tags_list:
  categorical_list = [0] * len(encode_tags_types)
  for Tags in revised_tags:
    tags_type_index = encode_tags_types[Tags]
    categorical_list[tags_type_index] = 1
  categorical_tags_list.append(categorical_list)

df['tags_cat_list'] = categorical_tags_list
df.shape

(20262, 6)

In [14]:
labels = list(encode_tags_types.keys())
len(labels), labels[:5]

(36, ['java', 'angular', 'python', 'python-3.x', 'javascript'])

# Data Split

In [15]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(18236, 2026)

In [16]:
valid_df = df.loc[valid_ids]
valid_df.head()

,Question,Summary,Tags,Username,revised_tags,tags_cat_list
17100,I hosted my website on Azure Linux VM but I can’t access the website with the designated ip on chrome it doesn’t not respond,"I'm hosting my website on an Azure Linux VM, but I can't access it using the ip given. I have tried adding inbound rules, but there was no result. I also tried allowing the VM to listen to the ...","['linux', 'azure', 'azure-virtual-machine']",Osas nathaniel,[azure],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
15696,How am i could get typecheck by extend parametrs from array of functions in typescript?,I'm trying to create function which take parameters from funcArray argument parameters and use like its own. I want to use this argument as plugins with additional parameters. How am i able to get ...,"['typescript', 'functional-programming']",Yaroslav Prokopchuk,[typescript],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
5438,Running django application on Compute Engine machine and expose it to the internet,I have a Django application that is running on port 8000 on a GCE instance like this: gunicorn --bind 0.0.0.0:8000 AImage.wsgi:application [2024-01-05 03:59:35 +0000] [158846] [INFO] Starting ...,"['django', 'nginx']",Lê Hồng Nhật,[django],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
14425,What's the best practice for authentication when creating a new SPA using React and .NET 8 api as backend,"Prerequisites: I would like to use a third party like Okta/Auth0/Userfront The problem is that most examples using these providers is storing the accesstoken in a cookie, which is readable (not ...","['reactjs', 'web', 'asp.net-web-api']",Timeshift,[reactjs],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10494,RELEASE build APK does not build correctly INTERMITANT,Soo Im going to struggle to explain this problem... but basically the way I can tell that this is occurring is that when I release/build my android app one of the first things the application does is ...,"['c#', 'android', 'visual-studio', 'build', 'apk']",TryingAgain,"[c#, android]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


# Fastai & Blurr Inference

In [17]:
model_path = "/content/drive/MyDrive/Capstone_3/models/Question-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [18]:
learner_inf.blurr_predict("random placeholder")

[{'labels': [],
  'scores': [],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['java', 'angular', 'python', 'python-3.x', 'javascript', 'html', 'css', 'react-native', 'c#', 'ios', 'r', 'android', 'django', 'azure', 'reactjs', 'flutter', 'kotlin', 'laravel', 'c++', 'postgresql', 'c', 'sql', 'mysql', 'docker', 'swift', 'visual-studio-code', 'excel', 'typescript', 'pandas', 'node.js', 'spring-boot', 'next.js', 'php', '.net', 'amazon-web-services', 'dart'],
  'probs': [0.00961464922875166,
   0.018605461344122887,
   0.006898907013237476,
   0.0017704714555293322,
   0.040572475641965866,
   0.014407854527235031,
   0.017226358875632286,
   0.1428435742855072,
   0.0011499017709866166,
   0.02255135215818882,
   0.005853455979377031,
   0.29825109243392944,
   0.0027830321341753006,
   0

In [19]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

[]


## Evaluation


In [20]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['tags_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [21]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['Summary']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_tags = [0] * len(encode_tags_types)
  for Tags in labels:
    pred_tags[encode_tags_types[Tags]] = 1
  preds.append(pred_tags)

preds[0][:20]

  0%|          | 0/2026 [00:00<?, ?it/s]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

In [22]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.614716825134353
F1 Score (Macro) = 0.5639640514284195


# Convert to ONNX

### ONNX
ONNX (Open Neural Network Exchange) is a open standard for representing machine learning models.
- It allows developers to move models between different frameworks (such as PyTorch, TensorFlow, and Caffe2) without losing performance or accuracy.
ONNX makes it easier to build and run AI models on a variety of hardware, including GPUs, CPUs, and custom accelerators.
- This standard helps eliminate the need for multiple conversion tools and provides a unified representation of the model across different tools and frameworks.
- ONNX is being developed by a collaboration of companies including Microsoft, Facebook, Amazon, and IBM, among others, making it a well-supported and widely-adopted standard.
- Converting to ONNX runtime often makes model faster.

In [23]:
model_path = "/content/drive/MyDrive/Capstone_3/models/Question-classifier-stage-1.pkl"
learner_inf = load_learner(model_path)

In [24]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/Question-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [25]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = '/content/drive/MyDrive/Capstone_3/models/Question-classifier.onnx'
quantized_onnx_model_path = '/content/drive/MyDrive/Capstone_3/models/Question-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

# ONNX Inference

## Normal ONNX

In [26]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_tags_types.keys())

inf_session = rt.InferenceSession('/content/drive/MyDrive/Capstone_3/models/Question-classifier.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [28]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Summary']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_tags = [0] * len(encode_tags_types)
  for Tags in labels:
    pred_tags[encode_tags_types[Tags]] = 1
  preds.append(pred_tags)

  0%|          | 0/2026 [00:00<?, ?it/s]

In [29]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6159868149979398
F1 Score (Macro) = 0.565016390344778


## Quantized ONNX

In [30]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_tags_types.keys())

inf_session = rt.InferenceSession('/content/drive/MyDrive/Capstone_3/models/Question-classifier-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [32]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['Summary']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_genres = [0] * len(encode_tags_types)
  for Tags in labels:
    pred_tags[encode_tags_types[Tags]] = 1
  preds.append(pred_tags)

  0%|          | 0/2026 [00:00<?, ?it/s]

In [33]:
metric_measures(valid_df, preds) #

F1 Score (Micro) = 0.0768248044211584
F1 Score (Macro) = 0.0686715954457695
